In [1]:
#default_exp process
%load_ext autoreload
%autoreload 2

In [2]:
#export
from pathlib import Path
from datetime import datetime
import pandas as pd
import pandas_read_xml as pdx
from anatelspyder.query import *

ModuleNotFoundError: No module named 'anatelspyder'

In [ ]:
#export
def read_stel(update=False):
    """Read the local Stel Database file and optionally update it."""
    if update:
        update_stel()
    if (file := Path('anatelspyder/files/stel.fth')).exists():
        stel = pd.read_feather(file)
    elif (file := Path('anatelspyder/files/stel.csv')).exists():
        stel = pd.read_csv(file)
    elif (file := Path('anatelspyder/files/Base_de_Dados.xlsx')).exists():
        stel = pd.read_excel(file, sheet_name='Stel', engine='openpyxl')
    else:
        update_stel()
        try:
            stel = pd.read_feather(Path('anatelspyder/files/stel.fth'))        
        except FileNotFoundError as e:
            raise ConnectionError("Base de Dados do Stel inexistente e não foi possível atualizá-la" ) from e
    return stel

In [ ]:
#export
def read_radcom(update=False):
    """Read the local Radcom Database file and optionally update it."""
    if update:
        update_radcom()
    if (file := Path('anatelspyder/files/radcom.fth')).exists():
        radcom = pd.read_feather(file)
    elif (file := Path('anatelspyder/files/radcom.csv')).exists():
        radcom = pd.read_csv(file)
    elif (file := Path('anatelspyder/files/Base_de_Dados.xlsx')).exists():
         radcom = pd.read_excel(file, sheet_name='Radcom', engine='openpyxl')
    else:
        update_radcom()
        try:
            radcom = pd.read_feather(Path('anatelspyder/files/radcom.fth'))        
        except FileNotFoundError as e:
            raise ConnectionError("Base de Dados do Stel inexistente e não foi possível atualizá-la" ) from e
    return radcom

In [ ]:
#export
def parse_estações(row, cols=COL_ESTACOES):
    """Receives a json row and filter the column in `cols`"""
    dict_result = {k: row[k] for k in cols}
    dict_result['Número da Estação'] = row['administrativo']['@numero_estacao']
    return dict_result

def parse_plano_basico(row, cols=COL_PLANO_BASICO):
    """Receives a json row and filter the column in `cols`"""
    if '@Frequencia' not in row:
        cols = list(cols)
        cols[cols.index('@Frequencia')] = '@Frequência'
    return {k: row[k] for k in cols}

def read_estações(path, cols):
    df = pdx.read_xml(path, cols)
    columns = ['Serviço', 'Situação', 'Entidade', 'Fistel', 'CNPJ', 'Município', 'UF', 'Id', 'Número da Estação']
    dtypes = ['category', 'category', 'str', 'str', 'str', 'str', 'category', 'str', 'str']
    df = pd.DataFrame(df['row'].apply(parse_estações).tolist())
    df.columns = columns
    for col, dtype in zip(columns, dtypes):
        df[col] = df[col].astype(dtype)
    return df

def read_plano_basico(path, cols):
    df = pdx.read_xml(path, cols)
    columns = ['Id', 'Classe', 'Frequência', 'Latitude', 'Longitude']
    dtypes = ['str', 'category', 'str', 'str', 'str']
    df = pd.DataFrame(df['row'].apply(parse_plano_basico).tolist())
    df.loc[df['@Frequencia'].isna(), '@Frequencia'] = df.loc[df['@Frequência'].notna(), '@Frequência']
    df.drop('@Frequência', axis=1, inplace=True)
    df.columns = columns
    for col, dtype in zip(columns, dtypes):
        df[col] = df[col].astype(dtype)
    return df

In [ ]:
#export
def read_mosaico(update=False):
    if update: 
        update_mosaico()
        estações = read_estações('anatelspyder/files/estações.zip', ['estacao_rd'])
        plano_basico = read_plano_basico('anatelspyder/files/plano_basico.zip', ['plano_basico'])
        df = pd.merge(estações, plano_basico, on='Id')
        df.to_feather('anatelspyder/files/mosaico.fth')
        return df
    if not (file := Path('anatelspyder/files/mosaico.fth')).exists():
            return read_mosaico(update=True)
    return pd.read_feather('anatelspyder/files/mosaico.fth')

In [ ]:
from nbdev.export import notebook2script; notebook2script()